In [52]:
import scipy
from scipy.integrate import quad
import numpy as np
from numpy.linalg import inv, eig, qr, det
import math
import random

In [53]:
n = 100
p = 10

X = np.random.rand(n,p)

for count in range(n):
    X[count][0] = 1

L = [[0 for a in range(p)] for b in range(p)]
L[0][0]=1

for i in range(1,p):
    L[i][i] = 1 #random.random()
    
#L = np.matrix('1 0; 0 0.5')
    
#B = np.random.rand(p)
B = [0.01 for count in range(p)]
#t_0 = 0.3


w = 5
y = X @ B + np.random.rand(n)/1000
#y = np.random.rand(n)

#y = np.array([1,0,2])

In [91]:
## Configuration for normalization and inverse CDF sampling
integration_config = {"NORM_STEP":0.01, "NORM_THRESHOLD":0.000001, "initial":0.001}

In [54]:
cdfMap = {}

In [55]:
## QR Decomposition of X

Q, R = qr(X)
#q = (t_0 ** 2) * (Q @ inv(R@R @ L @ L @ R.T) @ Q.T)

In [56]:
## Eigendecomposition of R(L^2)R^T

D_arr, V = eig(R @ L @ L @ R.T)
D = np.diag(D_arr) # Converting eigenvalues into eigenvalue matrix
d = 1 # Product of eigenvalues
for ele in D_arr:
    d = d*ele

In [57]:
## Storing important values
yTy = y.T @ y
yTQ = y.T @ Q
yTQQTy = yTQ @ yTQ.T
yTQV = y.T @ Q @ V
VTQTy = yTQV.T

In [58]:
## Optimized probability function

def probability(tau):
    ## Determinant
    det = 1
    for d_i in D_arr:
        det *= (1+(tau**2)*d_i)

    ##Difficult multiplication term
    mult = yTy - yTQQTy + (yTQV @ np.diag(1/(np.diag(D)+tau**(-2))) @ VTQTy)/(tau**2)

    return det**(-0.5) * ((w/2 + mult/2)**(-0.5*(n+w)))*(1/((tau**(-1))*(1+tau**(-2))))


In [59]:
## Unoptimized probability

def unop_probability(tau):
    ## Determinant
    det = 1
    for d_i in D_arr:
        det *= (1+(tau**2)*d_i)
        
    ## REALLY difficult multiplication term
    mat = inv(np.identity(n)+(tau**2)*(X@L@L@X.T))
    return det**(-0.5) * ((w/2 + (y.T @ mat @ y)/2)**(-0.5*(n+w)))*(1/((tau**(-1))*(1+tau**(-2))))

In [97]:
## Approximating the integral of the probability density function over all positive reals
## Default values NORM_STEP = 0.1, NORM_THRESHOLD = 0.0001, initial = 0.001

def g(config):
    NORM_STEP = config["NORM_STEP"]
    NORM_THRESHOLD = config["NORM_THRESHOLD"]
    initial = config["initial"]
    
    ## Initialize lower and upper vals
    lower = initial
    upper = lower + NORM_STEP
    prob = lambda x: probability(x)
    val = (prob(lower) + prob(upper))/2 * NORM_STEP
    
    cdfMap[(lower+upper)/2] = val
    
    ## Res is ultimately returned
    res = 0
    
    ## While there's a lot of change to res
    while res == 0 or (abs((val-res)/res) > NORM_THRESHOLD):
        res = val
        lower = lower + NORM_STEP
        upper = upper + NORM_STEP
        val += (prob(lower) + prob(upper))/2 * NORM_STEP
        cdfMap[val] = (lower+upper)/2
        #print(lower)
        #print(upper)
        #print(round((lower+upper)/2,5))
        #print("-----------------------------")
    return res, upper
    
    

In [93]:
g(integration_config)

(2.0139426924363208e-25, 1.9710000000000014)

In [83]:
## Inverse CDF of x

def F_inv(x, config, c):
    NORM_STEP = config["NORM_STEP"]
    NORM_THRESHOLD = config["NORM_THRESHOLD"]
    initial = config["initial"]
    find = NORM_STEP*int(x/NORM_STEP) + NORM_STEP/2.0 + initial
    return cdfMap[find]/c
    
    '''
    if x > 1:
        return math.inf
    
    ## Initializing lower and upper bounds
    lower = initial
    upper = initial + NORM_STEP
    
    ## Calling normalizing constant
    scal = g(NORM_STEP, NORM_THRESHOLD, initial)[0]
    prob = lambda x: probability(x)
    res = (prob(lower) + prob(upper))/2 * NORM_STEP
    
    ## While we're less than the normalizing constant times x, increment the result
    while res < x*scal:
        lower = lower + NORM_STEP
        upper = upper + NORM_STEP
        res += (prob(lower) + prob(upper))/2 * NORM_STEP
        
    return upper
    '''
    

In [96]:
config = integration_config
NORM_STEP = config["NORM_STEP"]
NORM_THRESHOLD = config["NORM_THRESHOLD"]
initial = config["initial"]
x = 0.7812241
NORM_STEP*int(x/NORM_STEP) + NORM_STEP/2.0 + initial

0.786

In [71]:
NORM_STEP = integration_config["NORM_STEP"]
NORM_THRESHOLD = integration_config["NORM_THRESHOLD"]
initial = integration_config["initial"]
NORM_STEP*int(0.5/NORM_STEP) + NORM_STEP/2.0
print(list(cdfMap.keys()))

[0.051000000000000004, 0.15100000000000002, 0.251, 0.35100000000000003, 0.451, 0.5509999999999999, 0.651, 0.7509999999999999, 0.851, 0.9509999999999998, 1.051, 1.151, 1.2510000000000001, 1.3510000000000002, 1.4510000000000003, 1.5510000000000004]


In [ ]:
## Plotting optimized probability

g_val = g()[0]

import matplotlib.pyplot as plt
a = [(i+0.01)/1000.0 for i in range(5,1000)]
b = [probability(j)/g_val for j in a]
plt.scatter(a,b)
plt.show()

In [ ]:
probability(0.09)

In [ ]:
## Plotting unoptimized probability

g_val = g()[0]

import matplotlib.pyplot as plt
a = [(i+0.01)/1000.0 for i in range(5,1000)]
b = [unop_probability(j)*(10**(35)) for j in a]
plt.scatter(a,b)

In [ ]:
x = np.random.rand(1000)
y = [F_inv(i) for i in x]
plt.hist(y)
plt.show()